In [ ]:
# wanted to start from scratch 
import polars as pl

df = pl.read_csv('2022_place_canvas_history.csv') 

In [ ]:
# restricting size of user_id for memory usage, should be lossless 
import base64 

def hashbase64(user_id):   
    return base64.b64encode(user_id.encode()).decode()[:16]

df = df.with_columns(pl.col('user_id').map_elements(hashbase64, return_dtype=pl.String))

In [ ]:
# no UTC 
df = df.with_columns(pl.col('timestamp').str.replace(r" UTC", "", literal=False))

In [ ]:
# normalizing milisecond precision
df = df.with_columns(
    pl.when(pl.col('timestamp').str.len_chars() == 22).then(pl.concat_str(pl.col('timestamp'), pl.lit('0')))
    .when(pl.col('timestamp').str.len_chars() == 21).then(pl.concat_str(pl.col('timestamp'), pl.lit('00')))
    .when(pl.col('timestamp').str.len_chars() == 19).then(pl.concat_str(pl.col('timestamp'), pl.lit('.000')))
    .otherwise(pl.col('timestamp'))
)

In [ ]:
# convert to datetime 
df = df.with_columns(pl.col('timestamp').str.to_datetime())

In [ ]:
# splitting coords 
df = df.with_columns(
        pl.col('coordinate').str.split(',').list.get(0).cast(pl.Int32).alias('x'),
        pl.col('coordinate').str.split(',').list.get(1).cast(pl.Int32).alias('y')
    ).drop('coordinate')

In [ ]:
# converting user_ids to ints 
df = df.with_columns(pl.col('user_id').rank("dense"))

In [12]:
df.write_parquet('2022_place.parquet')